In [1]:

#Conexión a la base de datos y carga de datos

import pandas as pd
import mysql.connector

# Detalles de conexión a la base de datos
user = 'root'
password = ''  # Deja la contraseña vacía si no tienes
host = 'localhost'
port = '3306'
database = 'eyc_tarea1'

# Conectar a la base de datos
connection = mysql.connector.connect(
    user=user,
    password=password,
    host=host,
    port=port,
    database=database
)

# Verificar la conexión
if connection.is_connected():
    print("Conexión exitosa a la base de datos")

# Cargar los datos de las tablas en DataFrames
edificios_df = pd.read_sql_query("SELECT * FROM edificios_escolares", connection)
colonias_df = pd.read_sql_query("SELECT * FROM cat_colonias", connection)

#print(edificios_df.head())
#print(colonias_df.head())


Conexión exitosa a la base de datos


C:\Users\cecir\AppData\Local\Temp\ipykernel_23292\3356444260.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  edificios_df = pd.read_sql_query("SELECT * FROM edificios_escolares", connection)
C:\Users\cecir\AppData\Local\Temp\ipykernel_23292\3356444260.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  colonias_df = pd.read_sql_query("SELECT * FROM cat_colonias", connection)


In [2]:
#Conversión de coordenadas y preparación de datos

# Convertir las coordenadas a tipo float
edificios_df['latitud'] = pd.to_numeric(edificios_df['latitud'], errors='coerce')
edificios_df['longitud'] = pd.to_numeric(edificios_df['longitud'], errors='coerce')
colonias_df['lat'] = pd.to_numeric(colonias_df['lat'], errors='coerce')
colonias_df['lon'] = pd.to_numeric(colonias_df['lon'], errors='coerce')

# Unir los DataFrames por código postal
merged_df = pd.merge(edificios_df, colonias_df, left_on='codigo_postal', right_on='cve_codpost', how='left')


In [3]:

#Cálculo de distancias utilizando la fórmula de Haversine

from math import radians, cos, sin, sqrt, atan2

# Función para calcular la distancia utilizando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en km
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

# Calcular las distancias utilizando la función de Haversine
merged_df['distancia_km'] = merged_df.apply(lambda row: haversine(row['latitud'], row['longitud'], row['lat'], row['lon']), axis=1)


In [ ]:
#Filtrado y visualización de resultados

# Filtrar los edificios que estén a 10 km o más de distancia
result_df = merged_df[merged_df['distancia_km'] >= 10000]

# Seleccionar columnas relevantes para mostrar los resultados
result_df_filtered = result_df[['cct', 'nombre', 'codigo_postal', 'latitud', 'longitud', 'nombre_colonia', 'distancia_km']]

# Mostrar los resultados filtrados

print(result_df_filtered)
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)
result_df_filtered

result_df_filtered.shape



In [13]:
from IPython.display import display, HTML

def make_clickable(link):
    return f'<a href="{link}" target="_blank">{link}</a>'

result_df_filtered['google_maps_link_clickable'] = result_df_filtered['google_maps_link'].apply(make_clickable)

# Mostrar los resultados con los enlaces clicables
display(HTML(result_df_filtered[['nombre', 'codigo_postal', 'distancia_km', 'google_maps_link_clickable']].to_html(escape=False)))


C:\Users\cecir\AppData\Local\Temp\ipykernel_23292\1637752325.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df_filtered['google_maps_link_clickable'] = result_df_filtered['google_maps_link'].apply(make_clickable)


,nombre,codigo_postal,distancia_km,google_maps_link_clickable
12,COORDINACION ESTATAL COAHUILA,00000,11110.880987,"https://www.google.com/maps/dir/25.433333333333,-101.0/0.0,0.0/"
21,SUBDIRECCION DE SERVICIOS EDUCATIVOS REGION LAGUNA,00000,11353.747730,"https://www.google.com/maps/dir/25.544444444444,-103.4416666666/0.0,0.0/"
24,DIRECCION DE ENLACE Y VINCULACION CON IES,00000,11106.726341,"https://www.google.com/maps/dir/25.428507,-100.958039/0.0,0.0/"
52,PERSONAL COMISIONADO A LA SEPC,00000,11110.880987,"https://www.google.com/maps/dir/25.433333333333,-101.0/0.0,0.0/"
56,DIRECCION DE SERVICIO COMUNITARIO,00000,11106.726341,"https://www.google.com/maps/dir/25.428507,-100.958039/0.0,0.0/"
68,DIRECCION DE ACCIONES COMPENSATORIAS,00000,11100.347248,"https://www.google.com/maps/dir/25.456826,-100.896892/0.0,0.0/"
74,PROGRAMA PIENSO CIENCIA Y TECNOLOGIA PARA NIÑOS,00000,11110.880987,"https://www.google.com/maps/dir/25.433333333333,-101.0/0.0,0.0/"
75,PROGRAMA PIENSO EN LA EDUCACION DE LOS NIÑOS Y NIÑAS DE COAHUILA,00000,11110.880987,"https://www.google.com/maps/dir/25.433333333333,-101.0/0.0,0.0/"
76,DIRECCION DE CALIDAD EDUCATIVA,00000,11110.880987,"https://www.google.com/maps/dir/25.433333333333,-101.0/0.0,0.0/"
77,DIRECCION DE INNOVACION EDUCATIVA,00000,11110.880987,"https://www.google.com/maps/dir/25.433333333333,-101.0/0.0,0.0/"
